In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from modeling_llama import LlamaForCausalLM
from transformers import AutoTokenizer
from datasets import load_dataset

In [ ]:
torch.nn.Linear.reset_parameters = lambda x: None
model = LlamaForCausalLM.from_pretrained('../llama-2-13b/', torch_dtype=torch.bfloat16)

In [ ]:
model = model.to('cuda:0').eval()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('../llama-2-13b/')

In [ ]:
prompts = []

In [ ]:
xsum = load_dataset('xsum').shuffle(4242)
cnn = load_dataset('cnn_dailymail', '3.0.0').shuffle(4242)

In [ ]:
for i in range(4):
    item = cnn['train'][i+100]
    cnn_context = 'Article: ' + item['article'] + '\nSummary: ' + item['highlights'].replace('\n', '')
    
    item = cnn['train'][i]
    prompt = cnn_context + '\nArticle: ' + item['article'] + '\nSummary:'
    prompts.append(prompt)

In [ ]:
for i in range(4):
    item = xsum['train'][i+100]
    xsum_context = 'Article: ' + item['document'] + '\nSummary: ' + item['summary'].replace('\n', '')
    
    item = xsum['train'][i]
    prompt = xsum_context + '\nArticle: ' + item['document'] + '\nSummary:'
    prompts.append(prompt)

In [ ]:
from searching import LayerSkippingSearching

In [ ]:
layer_searching = LayerSkippingSearching(model, tokenizer, prompts, evaluate_config={"generate_fn": "essg", "max_new_tokens": 32})

In [ ]:
layer_searching.probe([8,10,15,18,20,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,], [])
layer_searching.probe([3, 5, 6, 8, 10, 11, 14, 15, 18, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37], [6, 9, 10, 11, 15, 24, 25, 27, 28, 35])

In [ ]:
layer_searching.search(1000)

In [ ]:
layer_searching.get_solution()